# **Bitcoin price prediction - Random Forest Regressor**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: executing the chosen model, first with default values, then by choosing the best parameters by performing hyperparameter tuning with cross validation and performance evaluation.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = True # Decide whether or not to use operations that might slow down notebook execution
SPLITTING_METHOD = "walk_forward_splits"
MODEL_NAME = "RandomForestRegressor"
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING: 
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

In [3]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets names
DATASET_TRAIN_VALID_NAME = "bitcoin_blockchain_data_15min_train_valid"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features labels
FEATURES_LABEL = "features"
TARGET_LABEL = "next-market-price"

# Features names
ALL_FEATURES_NAME = "all_features"
MOST_REL_FEATURES_NAME = "most_rel_features"
LEAST_REL_FEATURES_NAME = "least_rel_features"

# Features paths
ALL_FEATURES = FEATURES_DIR + "/" + ALL_FEATURES_NAME + ".json"
MOST_REL_FEATURES = FEATURES_DIR + "/" + MOST_REL_FEATURES_NAME + ".json"
LEAST_REL_FEATURES = FEATURES_DIR + "/" + LEAST_REL_FEATURES_NAME + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

#####################
# --- UTILITIES --- #
#####################

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + ".csv"
MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [4]:
# Suppression of warnings for better reading
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
if not LOCAL_RUNNING:
    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

# Import files

In [6]:
# Import my files
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
import utilities, parameters

importlib.reload(utilities)
importlib.reload(parameters)

<module 'parameters' from 'D:\\Documents/Repository/BDC/project/utilities\\parameters.py'>

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
def dataset_info(dataset):
  # Print dataset
  dataset.show(3)

  # Get the number of rows
  num_rows = dataset.count()

  # Get the number of columns
  num_columns = len(dataset.columns)

  # Print the shape of the dataset
  print("Shape:", (num_rows, num_columns))

  # Print the schema of the dataset
  dataset.printSchema()

In [10]:
if SLOW_OPERATIONS:
  dataset_info(df)

+-------------------+---+------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+-----------------+--------------------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          timestamp| id|      market-price|      total-bitcoins|          market-cap|        trade-volume|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|   n-transactions|estimated-transaction-volume-usd|      rate-of-change|        sma-5-days|        sma-7-days|       sma-10-days|       sma-20-days|       sma-50-days|      sma-100-days| next-market-price|
+-----

# Loading features

In [11]:
# Loading all the features
with open(ALL_FEATURES, "r") as f:
    ALL_FEATURES = json.load(f)
print(ALL_FEATURES)

['market-price', 'market-cap', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days', 'trade-volume', 'sma-100-days', 'n-unique-addresses', 'transaction-fees-usd', 'total-bitcoins', 'n-transactions-total', 'blocks-size', 'sma-50-days', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions', 'rate-of-change']


In [12]:
# Loading the most relevant features
with open(MOST_REL_FEATURES, "r") as f:
    MOST_REL_FEATURES = json.load(f)
print(MOST_REL_FEATURES)

['market-price', 'market-cap', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days', 'trade-volume']


In [13]:
# Loading least relevant features
with open(LEAST_REL_FEATURES, "r") as f:
    LEAST_REL_FEATURES = json.load(f)
print(LEAST_REL_FEATURES)

['sma-100-days', 'n-unique-addresses', 'transaction-fees-usd', 'total-bitcoins', 'n-transactions-total', 'blocks-size', 'sma-50-days', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions', 'rate-of-change']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- **Default without normalization:** Make predictions using the chosen base model
- **Default with normalization:** Like the previous one but features are normalized

At this point, the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:

- **Hyperparameter tuning:** Researching the best parameters to use
- **Cross Validation:** Validate the performance of the model with the chosen parameters

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved to later make predictions on the test set.

---

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem):

- **Block time series splits:** Involves dividing the time series into blocks of equal length, and then using each block as a separate fold for cross-validation.

    ![block-splits.png](https://drive.google.com/uc?id=1SPT133HO1VdWYZZv6GeknFY3xX3T2tvL)

- **Walk forward time series splits:** Involves using a sliding window approach to create the training and validation sets for each fold. The model is trained on a fixed window of historical data, and then validated on the next observation in the time series. This process is repeated for each subsequent observation, with the window sliding forward one step at a time. 

    ![walk-forward-splits.png](https://drive.google.com/uc?id=1SNdq-kjbv4MXtdBj3EOJ2dmQpbbPStJi)

- **Single time series split:** Involves dividing the time series considering as validation set a narrow period of time and as train set everything that happened before this period, in such a way as to best benefit from the trend in the short term.

    ![single-split.png](https://drive.google.com/uc?id=1SODyQLolK4zn9lFGnNaqnMBZrHn3OsVn)

In [14]:
# Get splitting parameters based on the choosen splitting method
splitting_info = parameters.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'walk_forward_splits',
 'min_obser': 20000,
 'sliding_window': 5000}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

### Without normalization

In [15]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [16]:
# Get default parameters
params = parameters.get_defaults_model_params(MODEL_NAME)
params

{'numTrees': [20], 'maxDepth': [5], 'seed': [42]}

In [17]:
# Make predictions by using all the features
default_res_all, default_pred_all = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, ALL_FEATURES, ALL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_all

{'Model': 'RandomForestRegressor', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 863.590210335848, 'MSE': 745788.0513879142, 'MAE': 740.2523985955457, 'MAPE': 0.07947170083392614, 'R2': 0.019237945829099656, 'Adjusted_R2': 0.01845255079072461, 'Time': 5.477662563323975}
{'Model': 'RandomForestRegressor', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 424.823714392795, 'MSE': 180475.18831049107, 'MAE': 205.85725374990346, 'MAPE': 0.02008863955203405, 'R2': 0.26037565718844047, 'Adjusted_R2': 0.2597833654224252, 'Time': 2.9910545349121094}
{'Model': 'RandomForestRegressor', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 641.849561210393

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,walk_forward_splits,all_features,1,"(20000, 5000)","[20, 5, 42]",863.590210,7.457881e+05,740.252399,0.079472,0.019238,0.018453,5.477663
1,RandomForestRegressor,default,walk_forward_splits,all_features,2,"(20000, 5000)","[20, 5, 42]",424.823714,1.804752e+05,205.857254,0.020089,0.260376,0.259783,2.991055
2,RandomForestRegressor,default,walk_forward_splits,all_features,3,"(20000, 5000)","[20, 5, 42]",641.849561,4.119709e+05,569.052506,0.049685,-0.214973,-0.215946,2.179537
3,RandomForestRegressor,default,walk_forward_splits,all_features,4,"(20000, 5000)","[20, 5, 42]",1760.563215,3.099583e+06,1140.604329,0.080979,-0.082773,-0.083640,1.941507
4,RandomForestRegressor,default,walk_forward_splits,all_features,5,"(20000, 5000)","[20, 5, 42]",6636.688045,4.404563e+07,5314.672854,0.231116,-1.750084,-1.752286,1.853943
5,RandomForestRegressor,default,walk_forward_splits,all_features,6,"(20000, 5000)","[20, 5, 42]",13846.578329,1.917277e+08,11501.440081,0.263624,-2.472686,-2.475466,1.893180
6,RandomForestRegressor,default,walk_forward_splits,all_features,7,"(20000, 5000)","[20, 5, 42]",4227.014037,1.786765e+07,3620.839229,0.064714,0.168563,0.167897,1.613122
7,RandomForestRegressor,default,walk_forward_splits,all_features,8,"(20000, 5000)","[20, 5, 42]",6503.283277,4.229269e+07,5396.967265,0.133826,0.445714,0.445270,1.568752
8,RandomForestRegressor,default,walk_forward_splits,all_features,9,"(20000, 5000)","[20, 5, 42]",1796.220297,3.226407e+06,1508.641203,0.044640,0.557429,0.557074,1.589787
9,RandomForestRegressor,default,walk_forward_splits,all_features,10,"(20000, 5000)","[20, 5, 42]",2101.928282,4.418103e+06,1760.309241,0.037252,0.515273,0.514885,1.832042


In [18]:
# Make predictions by using the most relevant features
default_res_most_rel, default_pred_most_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, MOST_REL_FEATURES, MOST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_most_rel

{'Model': 'RandomForestRegressor', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 317.9967233769482, 'MSE': 101121.91607847529, 'MAE': 254.22619409488487, 'MAPE': 0.02761911371913406, 'R2': 0.8670177968790799, 'Adjusted_R2': 0.8669113046243284, 'Time': 1.166672706604004}
{'Model': 'RandomForestRegressor', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 415.4527901400327, 'MSE': 172601.02083513804, 'MAE': 191.00814790385792, 'MAPE': 0.018561608126770705, 'R2': 0.29264561074087336, 'Adjusted_R2': 0.2920791607795047, 'Time': 1.2156269550323486}
{'Model': 'RandomForestRegressor', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE':

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,walk_forward_splits,most_rel_features,1,"(20000, 5000)","[20, 5, 42]",317.996723,1.011219e+05,254.226194,0.027619,0.867018,0.866911,1.166673
1,RandomForestRegressor,default,walk_forward_splits,most_rel_features,2,"(20000, 5000)","[20, 5, 42]",415.452790,1.726010e+05,191.008148,0.018562,0.292646,0.292079,1.215627
2,RandomForestRegressor,default,walk_forward_splits,most_rel_features,3,"(20000, 5000)","[20, 5, 42]",635.861025,4.043192e+05,556.950293,0.048551,-0.192407,-0.193362,1.405743
3,RandomForestRegressor,default,walk_forward_splits,most_rel_features,4,"(20000, 5000)","[20, 5, 42]",1810.946754,3.279528e+06,1182.761718,0.084028,-0.145633,-0.146550,1.550648
4,RandomForestRegressor,default,walk_forward_splits,most_rel_features,5,"(20000, 5000)","[20, 5, 42]",6355.752406,4.039559e+07,4962.134942,0.213706,-1.522185,-1.524205,1.590970
5,RandomForestRegressor,default,walk_forward_splits,most_rel_features,6,"(20000, 5000)","[20, 5, 42]",11008.563793,1.211885e+08,8314.183769,0.183462,-1.195037,-1.196795,1.334160
6,RandomForestRegressor,default,walk_forward_splits,most_rel_features,7,"(20000, 5000)","[20, 5, 42]",3859.800720,1.489806e+07,3261.093852,0.056966,0.306747,0.306192,1.274125
7,RandomForestRegressor,default,walk_forward_splits,most_rel_features,8,"(20000, 5000)","[20, 5, 42]",7581.883642,5.748496e+07,6118.520436,0.153076,0.246605,0.246001,1.193827
8,RandomForestRegressor,default,walk_forward_splits,most_rel_features,9,"(20000, 5000)","[20, 5, 42]",1609.659075,2.591002e+06,1281.722681,0.038170,0.644588,0.644304,1.185667
9,RandomForestRegressor,default,walk_forward_splits,most_rel_features,10,"(20000, 5000)","[20, 5, 42]",1647.972608,2.715814e+06,1369.718472,0.029650,0.702038,0.701799,1.234450


In [19]:
# Make predictions by using the least relevant features
default_res_least_rel, default_pred_least_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, LEAST_REL_FEATURES, LEAST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_least_rel

{'Model': 'RandomForestRegressor', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 2207.9593912520654, 'MSE': 4875084.673418192, 'MAE': 2008.02988221442, 'MAPE': 0.21714162700341164, 'R2': -5.411068197808595, 'Adjusted_R2': -5.416202186355388, 'Time': 1.2932803630828857}
{'Model': 'RandomForestRegressor', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 707.5959986405835, 'MSE': 500692.0972921647, 'MAE': 514.2443405487493, 'MAPE': 0.05254439245935604, 'R2': -1.0519389223384552, 'Adjusted_R2': -1.0535821166706585, 'Time': 1.3516392707824707}
{'Model': 'RandomForestRegressor', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE':

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,walk_forward_splits,least_rel_features,1,"(20000, 5000)","[20, 5, 42]",2207.959391,4.875085e+06,2008.029882,0.217142,-5.411068,-5.416202,1.293280
1,RandomForestRegressor,default,walk_forward_splits,least_rel_features,2,"(20000, 5000)","[20, 5, 42]",707.595999,5.006921e+05,514.244341,0.052544,-1.051939,-1.053582,1.351639
2,RandomForestRegressor,default,walk_forward_splits,least_rel_features,3,"(20000, 5000)","[20, 5, 42]",839.259974,7.043573e+05,733.982763,0.063659,-1.077272,-1.078935,1.637828
3,RandomForestRegressor,default,walk_forward_splits,least_rel_features,4,"(20000, 5000)","[20, 5, 42]",2315.708077,5.362504e+06,1628.112530,0.117656,-0.873276,-0.874776,1.430375
4,RandomForestRegressor,default,walk_forward_splits,least_rel_features,5,"(20000, 5000)","[20, 5, 42]",6991.177553,4.887656e+07,5763.664445,0.253472,-2.051713,-2.054157,1.263716
5,RandomForestRegressor,default,walk_forward_splits,least_rel_features,6,"(20000, 5000)","[20, 5, 42]",14644.424761,2.144592e+08,12415.105816,0.286879,-2.884411,-2.887522,1.401308
6,RandomForestRegressor,default,walk_forward_splits,least_rel_features,7,"(20000, 5000)","[20, 5, 42]",5335.338262,2.846583e+07,4686.260248,0.084931,-0.324603,-0.325664,1.259222
7,RandomForestRegressor,default,walk_forward_splits,least_rel_features,8,"(20000, 5000)","[20, 5, 42]",15988.609754,2.556356e+08,13278.849579,0.330602,-2.350349,-2.353032,1.240662
8,RandomForestRegressor,default,walk_forward_splits,least_rel_features,9,"(20000, 5000)","[20, 5, 42]",3077.456172,9.470736e+06,2711.865578,0.078971,-0.299116,-0.300156,1.262401
9,RandomForestRegressor,default,walk_forward_splits,least_rel_features,10,"(20000, 5000)","[20, 5, 42]",8073.228598,6.517702e+07,7537.417737,0.159109,-6.150821,-6.156548,1.378072


### With normalization

In [20]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [21]:
# Make predictions by using all the features
default_norm_res_all, default_norm_pred_all = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, ALL_FEATURES, ALL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_all

{'Model': 'RandomForestRegressor', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 698.500124177801, 'MSE': 487902.4234764034, 'MAE': 600.7056085917982, 'MAPE': 0.06507261961081592, 'R2': 0.3583751010851439, 'Adjusted_R2': 0.3578612873522792, 'Time': 1.7398724555969238}
{'Model': 'RandomForestRegressor', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 621.9270314091215, 'MSE': 386793.2323973623, 'MAE': 427.0629090887931, 'MAPE': 0.04359112152103769, 'R2': -0.5851580097740667, 'Adjusted_R2': -0.5864274055776895, 'Time': 1.6465115547180176}
{'Model': 'RandomForestRegressor', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 143

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default_norm,walk_forward_splits,all_features,1,"(20000, 5000)","[20, 5, 42]",698.500124,4.879024e+05,600.705609,0.065073,0.358375,0.357861,1.739872
1,RandomForestRegressor,default_norm,walk_forward_splits,all_features,2,"(20000, 5000)","[20, 5, 42]",621.927031,3.867932e+05,427.062909,0.043591,-0.585158,-0.586427,1.646512
2,RandomForestRegressor,default_norm,walk_forward_splits,all_features,3,"(20000, 5000)","[20, 5, 42]",1439.059873,2.070893e+06,1386.699638,0.123120,-5.107423,-5.112314,1.592790
3,RandomForestRegressor,default_norm,walk_forward_splits,all_features,4,"(20000, 5000)","[20, 5, 42]",2541.803951,6.460767e+06,1673.471089,0.119805,-1.256931,-1.258738,1.511244
4,RandomForestRegressor,default_norm,walk_forward_splits,all_features,5,"(20000, 5000)","[20, 5, 42]",7480.757597,5.596173e+07,6546.393472,0.295364,-2.494091,-2.496889,1.614422
5,RandomForestRegressor,default_norm,walk_forward_splits,all_features,6,"(20000, 5000)","[20, 5, 42]",15860.157308,2.515446e+08,13761.944375,0.321382,-3.556124,-3.559772,1.438349
6,RandomForestRegressor,default_norm,walk_forward_splits,all_features,7,"(20000, 5000)","[20, 5, 42]",4718.352877,2.226285e+07,4097.348288,0.072270,-0.035959,-0.036789,1.574032
7,RandomForestRegressor,default_norm,walk_forward_splits,all_features,8,"(20000, 5000)","[20, 5, 42]",4676.832042,2.187276e+07,3549.872614,0.084979,0.713337,0.713107,1.529181
8,RandomForestRegressor,default_norm,walk_forward_splits,all_features,9,"(20000, 5000)","[20, 5, 42]",12916.680157,1.668406e+08,10253.780248,0.302429,-21.885786,-21.904113,1.461627
9,RandomForestRegressor,default_norm,walk_forward_splits,all_features,10,"(20000, 5000)","[20, 5, 42]",8829.408729,7.795846e+07,8039.292024,0.170287,-7.553122,-7.559971,1.394956


In [22]:
# Make predictions by using the most relevant features
default_norm_res_most_rel, default_norm_pred_most_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, MOST_REL_FEATURES, MOST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_most_rel

{'Model': 'RandomForestRegressor', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 382.49949263294786, 'MSE': 146305.86186446252, 'MAE': 327.74800855054883, 'MAPE': 0.037980381458592026, 'R2': 0.8075978324506587, 'Adjusted_R2': 0.8074437566408095, 'Time': 1.1605265140533447}
{'Model': 'RandomForestRegressor', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 451.7225124530709, 'MSE': 204053.22825691482, 'MAE': 326.9150579038997, 'MAPE': 0.033392228639082255, 'R2': 0.16374801289333452, 'Adjusted_R2': 0.1630783416323882, 'Time': 1.120774745941162}
{'Model': 'RandomForestRegressor', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [20,

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,1,"(20000, 5000)","[20, 5, 42]",382.499493,1.463059e+05,327.748009,0.037980,0.807598,0.807444,1.160527
1,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,2,"(20000, 5000)","[20, 5, 42]",451.722512,2.040532e+05,326.915058,0.033392,0.163748,0.163078,1.120775
2,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,3,"(20000, 5000)","[20, 5, 42]",1205.661029,1.453619e+06,1162.094985,0.103502,-3.286973,-3.290406,1.241014
3,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,4,"(20000, 5000)","[20, 5, 42]",1981.013981,3.924416e+06,1448.130662,0.106296,-0.370911,-0.372009,1.208957
4,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,5,"(20000, 5000)","[20, 5, 42]",6921.415653,4.790599e+07,5654.302346,0.247760,-1.991114,-1.993509,1.287882
5,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,6,"(20000, 5000)","[20, 5, 42]",14764.156610,2.179803e+08,13054.652487,0.309668,-2.948188,-2.951350,1.349636
6,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,7,"(20000, 5000)","[20, 5, 42]",23029.171058,5.303427e+08,21403.660473,0.377913,-23.678482,-23.698244,1.162301
7,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,8,"(20000, 5000)","[20, 5, 42]",32781.819393,1.074648e+09,31853.591825,0.683853,-13.084282,-13.095561,1.222416
8,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,9,"(20000, 5000)","[20, 5, 42]",4355.431943,1.896979e+07,3865.172334,0.113038,-1.602115,-1.604199,1.147222
9,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,10,"(20000, 5000)","[20, 5, 42]",3306.280735,1.093149e+07,2843.105912,0.060287,-0.199336,-0.200296,1.176989


In [23]:
# Make predictions by using the least relevant features
default_norm_res_least_rel, default_norm_pred_least_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, LEAST_REL_FEATURES, LEAST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_least_rel

{'Model': 'RandomForestRegressor', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 1869.2717875993344, 'MSE': 3494177.015914811, 'MAE': 1667.7003629870912, 'MAPE': 0.1814637473567142, 'R2': -3.59508062832031, 'Adjusted_R2': -3.598760372567213, 'Time': 1.2435688972473145}
{'Model': 'RandomForestRegressor', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5, 42], 'RMSE': 1181.8048610755957, 'MSE': 1396662.729661908, 'MAE': 925.2914732494453, 'MAPE': 0.09555931127214234, 'R2': -4.723810365435916, 'Adjusted_R2': -4.728393997360189, 'Time': 1.3350560665130615}
{'Model': 'RandomForestRegressor', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [20, 5

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,1,"(20000, 5000)","[20, 5, 42]",1869.271788,3.494177e+06,1667.700363,0.181464,-3.595081,-3.598760,1.243569
1,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,2,"(20000, 5000)","[20, 5, 42]",1181.804861,1.396663e+06,925.291473,0.095559,-4.723810,-4.728394,1.335056
2,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,3,"(20000, 5000)","[20, 5, 42]",2189.417251,4.793548e+06,2041.056019,0.179638,-13.137003,-13.148324,1.362477
3,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,4,"(20000, 5000)","[20, 5, 42]",2979.407571,8.876869e+06,2171.580803,0.159246,-2.100944,-2.103428,1.249236
4,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,5,"(20000, 5000)","[20, 5, 42]",8944.052083,7.999607e+07,8259.943536,0.382429,-3.994727,-3.998726,1.356362
5,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,6,"(20000, 5000)","[20, 5, 42]",17805.853203,3.170484e+08,16154.437196,0.384284,-4.742567,-4.747166,1.236202
6,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,7,"(20000, 5000)","[20, 5, 42]",8852.049454,7.835878e+07,7790.290366,0.139439,-2.646276,-2.649195,1.406424
7,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,8,"(20000, 5000)","[20, 5, 42]",13007.502262,1.691951e+08,11672.206956,0.268790,-1.217463,-1.219239,1.257922
8,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,9,"(20000, 5000)","[20, 5, 42]",2878.895166,8.288037e+06,2482.909528,0.071597,-0.136883,-0.137793,1.289491
9,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,10,"(20000, 5000)","[20, 5, 42]",12583.396757,1.583419e+08,12183.609665,0.259722,-16.372295,-16.386207,1.399391


In [24]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [25]:
# Define the results to show
default_comparison_lst = [default_res_all, default_res_most_rel, default_res_least_rel, default_norm_res_all, default_norm_res_most_rel, default_norm_res_least_rel]

# Show the comparison table
default_comparison_lst_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,walk_forward_splits,all_features,"[20, 5, 42]",3633.810289,2.435331e+07,3053.893649,0.102658,-1.448513,-1.450474,1.967802
0,RandomForestRegressor,default,walk_forward_splits,most_rel_features,"[20, 5, 42]",3239.527915,1.902744e+07,2657.261563,0.090370,-1.345786,-1.347664,1.333877
0,RandomForestRegressor,default,walk_forward_splits,least_rel_features,"[20, 5, 42]",6060.488142,5.805877e+07,5312.267269,0.171499,-6.748423,-6.754628,1.312483
0,RandomForestRegressor,default_norm,walk_forward_splits,all_features,"[20, 5, 42]",5190.939166,4.457179e+07,4437.898290,0.144396,-4.181290,-4.185439,1.592361
0,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,"[20, 5, 42]",7259.576928,1.089601e+08,6644.448915,0.201235,-21.847933,-21.866230,1.215825
0,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,"[20, 5, 42]",7264.564725,7.955332e+07,6472.244594,0.217337,-13.098132,-13.109422,1.331579


In [40]:
# Save the best default model results and predicitons
best_default_results = default_res_most_rel
best_default_predictions = default_pred_most_rel

## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration. 

### Hyperparameter tuning

In [27]:
# Select the type of feature to be used
MODEL_TYPE = "hyp_tuning"
CHOSEN_FEATURES = MOST_REL_FEATURES
CHOSEN_FEATURES_LABEL = MOST_REL_FEATURES_NAME
FEATURES_NORMALIZATION = False

In [28]:
# Get model grid parameters
params = parameters.get_model_grid_params(MODEL_NAME)
params

{'numTrees': [3, 5, 10, 20, 30], 'maxDepth': [3, 5, 10], 'seed': [42]}

In [29]:
# Perform hyperparameter tuning
hyp_res = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
hyp_res

{'Model': 'RandomForestRegressor', 'Type': 'hyp_tuning', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [3, 10, 42], 'RMSE': 262.31110316697595, 'MSE': 68807.11484467589, 'MAE': 174.94483157135377, 'MAPE': 0.01963797666900897, 'R2': 0.9095139602048458, 'Adjusted_R2': 0.9094414989117166, 'Time': 1.0916557312011719}
{'Model': 'RandomForestRegressor', 'Type': 'hyp_tuning', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [3, 10, 42], 'RMSE': 383.7794961301079, 'MSE': 147286.70164987948, 'MAE': 173.974951958608, 'MAPE': 0.016826232767331913, 'R2': 0.3963888835219914, 'Adjusted_R2': 0.3959055112565435, 'Time': 1.069072961807251}
{'Model': 'RandomForestRegressor', 'Type': 'hyp_tuning', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [3, 10, 42], '

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,hyp_tuning,walk_forward_splits,most_rel_features,1,"(20000, 5000)","[3, 10, 42]",262.311103,6.880711e+04,174.944832,0.019638,0.909514,0.909441,1.091656
1,RandomForestRegressor,hyp_tuning,walk_forward_splits,most_rel_features,2,"(20000, 5000)","[3, 10, 42]",383.779496,1.472867e+05,173.974952,0.016826,0.396389,0.395906,1.069073
2,RandomForestRegressor,hyp_tuning,walk_forward_splits,most_rel_features,3,"(20000, 5000)","[3, 10, 42]",498.648826,2.486507e+05,434.055877,0.038468,0.266686,0.266099,1.180268
3,RandomForestRegressor,hyp_tuning,walk_forward_splits,most_rel_features,4,"(20000, 5000)","[3, 10, 42]",1548.579028,2.398097e+06,989.121098,0.070193,0.162276,0.161605,0.978908
4,RandomForestRegressor,hyp_tuning,walk_forward_splits,most_rel_features,5,"(20000, 5000)","[3, 5, 42]",6276.097155,3.938940e+07,4834.972349,0.207131,-1.459362,-1.461331,0.494623
5,RandomForestRegressor,hyp_tuning,walk_forward_splits,most_rel_features,6,"(20000, 5000)","[5, 10, 42]",10476.955960,1.097666e+08,7544.215008,0.163469,-0.988157,-0.989750,1.028033
6,RandomForestRegressor,hyp_tuning,walk_forward_splits,most_rel_features,7,"(20000, 5000)","[3, 10, 42]",3441.612455,1.184470e+07,2632.893608,0.046355,0.448830,0.448388,0.992996
7,RandomForestRegressor,hyp_tuning,walk_forward_splits,most_rel_features,8,"(20000, 5000)","[5, 3, 42]",6878.742277,4.731710e+07,5379.480904,0.135732,0.379864,0.379368,0.383769
8,RandomForestRegressor,hyp_tuning,walk_forward_splits,most_rel_features,9,"(20000, 5000)","[20, 10, 42]",1394.442713,1.944470e+06,1097.561446,0.032642,0.733274,0.733060,2.441995
9,RandomForestRegressor,hyp_tuning,walk_forward_splits,most_rel_features,10,"(20000, 5000)","[30, 10, 42]",1063.556141,1.131152e+06,788.650479,0.017018,0.875897,0.875798,3.658561


---
To select the best parameters to be used in the final model I assign a score to each value in the "Parameters" column based on the following criteria:
* Calculate the frequency of each unique value in the "Parameters" column.
* Normalize the frequencies to a scale of 0 to 1, where 1 represents the highest frequency.
* Calculate the split weight for each value in the "Parameters" column, where a higher split number corresponds to a higher weight.
* Normalize the split weights to a scale of 0 to 1, where 1 represents the highest split weight.
* Calculate the RMSE weight for each value in the "Parameters" column, where a lower RMSE value corresponds to a higher weight.
* Normalize the RMSE weights to a scale of 0 to 1, where 1 represents the highest RMSE weight.

Then calculate the overall score for each value in the "Parameters" column by combining the normalized frequency, split weight, and RMSE weight and take into consideration the parameters that have the highest score.

In [30]:
# Show parameters score
grouped_scores, best_params = parameters.choose_best_params(hyp_res)
grouped_scores

,Split weight,RMSE weight,Frequency weight,Final score
Parameters,,,,
"(30, 10, 42)",0.696970,0.664238,1.000000,0.462954
"(5, 10, 42)",0.772727,0.652665,0.666667,0.336221
"(20, 10, 42)",0.484848,0.787962,0.500000,0.191021
"(10, 10, 42)",0.772727,0.960191,0.166667,0.123661
"(3, 10, 42)",0.154545,0.882887,0.833333,0.113705
"(10, 5, 42)",0.636364,0.931118,0.166667,0.098755
"(5, 3, 42)",0.363636,0.343441,0.166667,0.020815
"(3, 5, 42)",0.227273,0.400962,0.166667,0.015188


In [31]:
# Print best parameters
print(f"Best parameters: {best_params}")

Best parameters: (30, 10, 42)


### Cross validation

In [32]:
MODEL_TYPE = "cross_val"

In [33]:
# Get tuned parameters
params = parameters.get_best_model_params(best_params, MODEL_NAME)
params

{'numTrees': [30], 'maxDepth': [10], 'seed': [42]}

In [34]:
# Perform cross validation
cv_res, cv_pred = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
cv_res

{'Model': 'RandomForestRegressor', 'Type': 'cross_val', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [30, 10, 42], 'RMSE': 320.68272014619896, 'MSE': 102837.40700036538, 'MAE': 266.2938609496898, 'MAPE': 0.029003499108288314, 'R2': 0.86476180954148, 'Adjusted_R2': 0.864653510690262, 'Time': 4.199690103530884}
{'Model': 'RandomForestRegressor', 'Type': 'cross_val', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [30, 10, 42], 'RMSE': 489.13799999380393, 'MSE': 239255.98303793854, 'MAE': 314.6874681083824, 'MAPE': 0.03172359710482812, 'R2': 0.01947990261283883, 'Adjusted_R2': 0.01869470133364992, 'Time': 4.208567380905151}
{'Model': 'RandomForestRegressor', 'Type': 'cross_val', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [30, 10, 42], '

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,1,"(20000, 5000)","[30, 10, 42]",320.682720,1.028374e+05,266.293861,0.029003,0.864762,0.864654,4.199690
1,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,2,"(20000, 5000)","[30, 10, 42]",489.138000,2.392560e+05,314.687468,0.031724,0.019480,0.018695,4.208567
2,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,3,"(20000, 5000)","[30, 10, 42]",521.865656,2.723438e+05,467.866890,0.041589,0.196811,0.196168,3.909075
3,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,4,"(20000, 5000)","[30, 10, 42]",1729.433993,2.990942e+06,1127.271506,0.080098,-0.044822,-0.045658,4.309117
4,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,5,"(20000, 5000)","[30, 10, 42]",6304.300731,3.974421e+07,4900.214503,0.210687,-1.481515,-1.483502,3.472915
5,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,6,"(20000, 5000)","[30, 10, 42]",10745.306908,1.154616e+08,8026.575559,0.176474,-1.091309,-1.092984,3.422320
6,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,7,"(20000, 5000)","[30, 10, 42]",3885.204010,1.509481e+07,3289.286367,0.058750,0.297592,0.297029,3.502126
7,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,8,"(20000, 5000)","[30, 10, 42]",7636.423515,5.831496e+07,6543.365356,0.160165,0.235727,0.235115,3.620844
8,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,9,"(20000, 5000)","[30, 10, 42]",1420.376620,2.017470e+06,1068.234982,0.032048,0.723261,0.723039,3.808050
9,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,10,"(20000, 5000)","[30, 10, 42]",1063.556141,1.131152e+06,788.650479,0.017018,0.875897,0.875798,3.788308


In [35]:
# Define the results to show
tuned_comparison_lst = [cv_res]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,"[30, 10, 42]",3037.411421,1.775607e+07,2487.27818,0.08331,-0.43656,-0.43771,3.90103


# Comparison table
Visualization of model performance at various stages of train / validation

In [36]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df], ignore_index=True))
final_comparison_lst_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,walk_forward_splits,all_features,"[20, 5, 42]",3633.810289,2.435331e+07,3053.893649,0.102658,-1.448513,-1.450474,1.967802
1,RandomForestRegressor,default,walk_forward_splits,most_rel_features,"[20, 5, 42]",3239.527915,1.902744e+07,2657.261563,0.090370,-1.345786,-1.347664,1.333877
2,RandomForestRegressor,default,walk_forward_splits,least_rel_features,"[20, 5, 42]",6060.488142,5.805877e+07,5312.267269,0.171499,-6.748423,-6.754628,1.312483
3,RandomForestRegressor,default_norm,walk_forward_splits,all_features,"[20, 5, 42]",5190.939166,4.457179e+07,4437.898290,0.144396,-4.181290,-4.185439,1.592361
4,RandomForestRegressor,default_norm,walk_forward_splits,most_rel_features,"[20, 5, 42]",7259.576928,1.089601e+08,6644.448915,0.201235,-21.847933,-21.866230,1.215825
5,RandomForestRegressor,default_norm,walk_forward_splits,least_rel_features,"[20, 5, 42]",7264.564725,7.955332e+07,6472.244594,0.217337,-13.098132,-13.109422,1.331579
6,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,"[30, 10, 42]",3037.411421,1.775607e+07,2487.278180,0.083310,-0.436560,-0.437710,3.901030


# Model accuracy

Since predicting the price accurately is very difficult let's se how good the model is at predicting whether the price will go up or down. 

For each row in our predictions let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
We compute whether the current prediction is correct (1) or not (0):

$$ 
prediction\_is\_correct
= 
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that we count the number of correct prediction:
$$ 
correct\_predictions
= 
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally we compute the percentage of accuracy of the model:
$$
\\ 
accuracy 
= 
(correct\_predictions / total\_rows) 
* 100
$$

In [37]:
# Convert the pandas dataset to a PySpark dataset
best_default_pred_spark = spark.createDataFrame(best_default_predictions)
validated_pred_spark = spark.createDataFrame(cv_pred)

# Compute model accuracy
default_accuracy = utilities.model_accuracy(best_default_pred_spark)
validated_accuracy = utilities.model_accuracy(validated_pred_spark)

# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy (default)': default_accuracy,
    'Accuracy (validated)': validated_accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (default): {default_accuracy:.2f}%")
print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (validated): {validated_accuracy:.2f}%")

Percentage of correct predictions for RandomForestRegressor with most_rel_features and walk_forward_splits (default): 47.88%
Percentage of correct predictions for RandomForestRegressor with most_rel_features and walk_forward_splits (validated): 47.89%


# Saving final results


In [41]:
# Concatenate default and tuned results
default_tuned_results = [best_default_results, cv_res]
default_tuned_results_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in default_tuned_results])
default_tuned_results_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,RandomForestRegressor,default,walk_forward_splits,most_rel_features,"[20, 5, 42]",3239.527915,1.902744e+07,2657.261563,0.09037,-1.345786,-1.347664,1.333877
0,RandomForestRegressor,cross_val,walk_forward_splits,most_rel_features,"[30, 10, 42]",3037.411421,1.775607e+07,2487.278180,0.08331,-0.436560,-0.437710,3.901030


In [42]:
# Saving default and tuned results
default_tuned_results_df.to_csv(MODEL_RESULTS, index=False)

In [43]:
# Saving accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)